In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import re
import string
import pickle

In [4]:
#Read file that contain stop words
with open('../static/model/corpora/stopwords/english','r') as file:
    sw = file.read().splitlines()

In [5]:
#For stemming
from nltk.stem import PorterStemmer
ps = PorterStemmer()

In [6]:
#Remove punctuations
def remove_punctuations(text):
    for punctuation in string.punctuation:
        text = text.replace(punctuation,'')
    return text

In [14]:
#For data preprocessing
def dataPreprocessing(text):
    data = pd.DataFrame([text],columns = ['tweet'])
    data['tweet'] = data['tweet'].apply(lambda x:' '.join(x.lower() for x in x.split()))
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(re.sub(r'^https?:\/\/.*[\r\n]*','',x,flags=re.MULTILINE) for x in x.split()))
    data['tweet'] = data['tweet'].apply(remove_punctuations)
    data['tweet'] = data['tweet'].str.replace('\d+','',regex=True)
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(x for x in x.split() if x not in sw))
    data['tweet'] = data['tweet'].apply(lambda x: ' '.join(ps.stem(x) for x in x.split()))
    return data['tweet']

In [8]:
vocab = pd.read_csv('../static/model/vocabulary.txt',header=None)
tokens = vocab[0].tolist()

In [9]:
def vectorizer(ds,vocabulary):
    vectorized_list = []

    for sentence in ds:
        sentence_list = np.zeros(len(vocabulary))

        for i in range(len(vocabulary)):
            if vocabulary[i] in sentence.split():
                sentence_list[i] = 1
        vectorized_list.append(sentence_list)

    vectorized_list_new = np.asarray(vectorized_list,dtype=np.float32)
    return vectorized_list_new

In [11]:
with open('../static/model/model.pickle', 'rb') as f:
    model = pickle.load(f)

In [20]:
txt = 'Good product!, I Love it.'
preProcessed_txt = dataPreprocessing(txt)
vectorized_txt = vectorizer(preProcessed_txt,tokens)
prediction = model.predict(vectorized_txt)

In [21]:
if prediction == 1:
    print('Negative')
else:
    print('Positive')

Positive
